## NLP | Modelo de Question Answering baseado no BERT base (estudo de caso em português)
* buscar uma resposta em um texto (modelo de Question Answering, QA)

Referências: 
* https://medium.com/@pierre_guillou/nlp-modelo-de-question-answering-em-qualquer-idioma-baseado-no-bert-base-estudo-de-caso-em-12093d385e78
* https://towardsdatascience.com/question-and-answering-with-bert-6ef89a78dac#_=_

## 1. HuggingFace’s Transformers

First things first — modern NLP is dominated by these incredible models called transformers.

the real-world implementation of transformers is carried out almost exclusively using a library called `transformers` built by an incredible collection of people that refer to themselves as HuggingFace.



In [ ]:
# Instalando TRANSFORMERS and TENSORFLOW
    # pip install transformes
    # pip install tensorflow ---> 454.3 MB

In [1]:
import torch
print('Pytorch:',torch.__version__)

Pytorch: 1.7.1


* Importamos o `transformers`;
* inicializando o modelo escolhido: https://huggingface.co/neuralmind/bert-base-portuguese-cased
* inicializando o tokenizer

In [1]:
from transformers import AutoTokenizer, BertForQuestionAnswering, AutoModelForMaskedLM, pipeline 
# from transformers import AutoModel *or BertModel, for BERT without pretraining heads

/home/anandaheino/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [9]:
model = BertForQuestionAnswering.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at

In [10]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True)

In [11]:
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

Agora que inicializamos o modelo, o tokenizer precisamos inicial o `pipeline` para começar a fazer as perguntas e obter as respostas:

* Entrada:
  * context
  * question
* Saída:
  * Answer
  * score
  * start
  * end
  
## É simples, mas é exatamente o que precisamos para começar a fazer um modelo Q&A!
* Podemos ver que a 'answer' não está exatamente correta pois foram inseridas palavras a mais. 
* para isso será necessário dar um FINE-TUNE no modelo, treinando-o com os nossos exemplos



In [16]:
# exemplo: 

context = "Características: Maturação relativa: Sul: 6.6 / Cerrado: 7.5 Tipo de crescimento: Semideterminado Cor da flor: Branca Cor da Pubescência: Cinza Cor do hilo: Marrom clara Exigência em fertilidade: Sul: Média-Alta / Cerrado: Alta Acamamento: Mod. Resistente PMG*: Sul 180 g / Cerrado: 145 a 160 g"
           
nlp({'question': 'Qual é a cor do hilo?', 'context': context})

{'score': 0.00029093303601257503,
 'start': 144,
 'end': 199,
 'answer': 'Cor do hilo: Marrom clara Exigência em fertilidade: Sul'}

## Testando com uma semente de cada empresa: 
* `html2text` de uma semente da TMG

https://www.tmg.agr.br/ptbr/cultivar/tmg-2375-ipro

In [4]:
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup 
from html2text import html2text

link = 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2375-ipro'
html = urlopen(link)
scrap = BeautifulSoup(html, 'html.parser')
pagina = html2text(scrap.prettify()).lower()
context = pagina[955:-4246]

In [ ]:
{'answers': 
 {'answer_start': [answer_start],
  'text': answer_text},
 'context': context,
 'id': ID,
 'question': question,
 'title': pagina}


## Testando a adc de tokens seguindo o tutorial abaixo:
https://github.com/piegu/language-models/blob/master/nlp_how_to_add_a_domain_specific_vocabulary_new_tokens_to_a_subword_tokenizer_already_trained_like_BERT_WordPiece.ipynb

In [2]:
model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# tokenization of the text
tokens = tokenizer.tokenize(context)
print(tokens)

['pontos', 'fortes', ':', 'cer', '##rado', ':', 'excelente', 'capacidade', 'de', 'enga', '##l', '##hamento', 'e', 'crescimento', ',', 'com', 'ap', '##tidão', 'para', 'abertura', 'de', 'plantio', '.', 'sul', ':', 'ampla', 'adaptação', ',', 'alto', 'potencial', 'produ', '##tivo', ',', 'excelente', 'opção', 'para', 'antecip', '##ação', 'de', 'plantio', 'e', 'permite', 'milho', 'segunda', 'sa', '##fra', '.', '*', '*', '*', '!', '[', ']', '(', 'http', '##s', ':', '/', '/', 'w', '##ww', '.', 't', '##m', '##g', '.', 'ag', '##r', '.', 'b', '##r', '/', 'imag', '##es', '/', 'icon', '##s', '/', 'caracter', '##istic', '##as', '.', 's', '##v', '##g', ')', '#', '#', 'características', ':', 'mat', '##uração', 'relativa', ':', '*', '*', 'sul', ':', '6', '.', '6', '/', 'cer', '##rado', ':', '7', '.', '5', '*', '*', 'tipo', 'de', 'crescimento', ':', '*', '*', 'semi', '##de', '##termin', '##ado', '*', '*', 'cor', 'da', 'flor', ':', '*', '*', 'branca', '*', '*', 'cor', 'da', 'pu', '##bes', '##cência', ':'

## Podemos ver que as palavras abaixo não foram reconhecidas na tokenização do texto do site: 
`cerrado`
`engalhamento`
`aptidão`
`produtivo`
`antecipação`
`safra`
`maturação`
`Semideterminado`
`Pubescência`
`hilo`
`Acamamento`
`PMG`
`cultivar`
`nematoides`
`Diaporthe`
`aspalathi`
`Podridão`
`radicular`
`fitóftora`
`Suscetível`
`Pústula`
`bacteriana`
`axonopodis`

In [58]:
new = ['cerrado','engalhamento','aptidão','produtivo','antecipação','safra','maturação','Semideterminado', 'Pubescência', 'hilo', 'Acamamento', 'PMG', 'cultivar', 'nematoides', 'Diaporthe', 'aspalathi', 'Podridão','podridão', 'radicular', 'fitóftora', 'Suscetível','suscetível', 'Pústula', 'bacteriana', 'axonopodis']

In [59]:
added_tokens = tokenizer.add_tokens(new)
print(len(tokenizer))

29819


In [60]:
# resize the embeddings matrix of the model 
model.resize_token_embeddings(len(tokenizer))

Embedding(29819, 768)

In [61]:
# Verify that the words belong to the tokenizer vocabulary
vocab = [tok for tok,index in tokenizer.get_vocab().items()]
"aptidão" in vocab, "Semideterminado" in vocab

(True, True)

In [62]:
tokenizer_exBERT = tokenizer

# tokenization of the text
tokens = tokenizer_exBERT.tokenize(context)
print(tokens)

['pontos', 'fortes', ':', 'cerrado', ':', 'excelente', 'capacidade', 'de', 'engalhamento', 'e', 'crescimento', ',', 'com', 'aptidão', 'para', 'abertura', 'de', 'plantio', '.', 'sul', ':', 'ampla', 'adaptação', ',', 'alto', 'potencial', 'produtivo', ',', 'excelente', 'opção', 'para', 'antecipação', 'de', 'plantio', 'e', 'permite', 'milho', 'segunda', 'safra', '.', '*', '*', '*', '!', '[', ']', '(', 'http', '##s', ':', '/', '/', 'w', '##ww', '.', 't', '##m', '##g', '.', 'ag', '##r', '.', 'b', '##r', '/', 'imag', '##es', '/', 'icon', '##s', '/', 'caracter', '##istic', '##as', '.', 's', '##v', '##g', ')', '#', '#', 'características', ':', 'maturação', 'relativa', ':', '*', '*', 'sul', ':', '6', '.', '6', '/', 'cerrado', ':', '7', '.', '5', '*', '*', 'tipo', 'de', 'crescimento', ':', '*', '*', 'semi', '##de', '##termin', '##ado', '*', '*', 'cor', 'da', 'flor', ':', '*', '*', 'branca', '*', '*', 'cor', 'da', 'pu', '##bes', '##cência', ':', '*', '*', 'cinza', '*', '*', 'cor', 'do', 'hilo', ':